In [1]:
import pandas as pd
import numpy as np
import random
import os
import sys
sys.path.append('../..')
from modules.many_features import utils, constants, env
from modules.many_features.random_agent import RandomAgent
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\User\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

In [3]:
#df = pd.read_csv('../../data/anemia_synth_dataset_some_nans_unspecified_more_feats.csv')
#df = pd.read_csv('../../data/more_features/more_feats_new_labels_0.1.csv')
df = pd.read_csv('../../data/more_features//more_feats_correlated_noisy_4_missing_1.csv')
df = df.fillna(-1)
df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,8.725667,-1.000000,2.344526,-1.000000,-1.000000,96.678418,-1.000000,2.707637,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,26.177001,-1.000000,Hemolytic anemia
1,12.430232,231.691743,5.143660,5.339158,484.590922,87.353532,-1.000000,4.268940,0,-1.000000,-1.000000,-1.000000,8.718062,-1.000000,-1.000000,37.290695,-1.000000,No anemia
2,6.737760,360.843434,-1.000000,5.481140,421.177091,75.086568,82.986344,2.691997,0,0.458698,19.596767,69.527433,-1.000000,-1.000000,-1.000000,20.213279,19.703433,Anemia of chronic disease
3,12.313034,-1.000000,2.765581,0.000000,-1.000000,99.494524,-1.000000,3.712677,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,36.939103,-1.000000,Unspecified anemia
4,6.758140,-1.000000,1.696525,-1.000000,-1.000000,86.643128,199.741346,2.339992,0,0.887158,80.614718,109.197814,18.770122,7.668335,63.159054,20.274421,-1.000000,Aplastic anemia


In [4]:
class_dict = constants.CLASS_DICT
df['label'] = df['label'].replace(class_dict)
X = df.iloc[:, 0:-1]
y = df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=SEED)
X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((56000, 17), (14000, 17), (56000,), (14000,))

#### The Random Agent

In [5]:
%%time
rand_agent = RandomAgent(X_test[0:], y_test[0:])
test_df = rand_agent.test()
test_df.head()

Testing done.....
Wall time: 3min 14s


,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred
0,2.0,0.0,0.0,-1.0,0.0,"[ret_count, Vitamin B12/Folate deficiency anemia]",0.0,1.0
1,1.0,1.0,1.0,1.0,0.0,[No anemia],0.0,0.0
2,2.0,2.0,1.0,1.0,0.0,"[segmented_neutrophils, Iron deficiency anemia]",4.0,4.0
3,1.0,3.0,0.0,-1.0,0.0,[Anemia of chronic disease],1.0,3.0
4,1.0,4.0,0.0,-1.0,0.0,[Anemia of chronic disease],6.0,3.0


In [6]:
# %%time
# rand_agent.test_sample(2)

In [7]:
# from datetime import datetime
# start = datetime.now()
# rand_agent.test_sample(2)
# end = datetime.now()
# duration = end-start
# duration

In [8]:
len(X_test), len(test_df)

(14000, 14000)

In [9]:
# test_df.to_csv('../../test_dfs/many_features/random_test_df_noisy_0.4.csv', index=False)

In [10]:
success_rate, success_df = utils.success_rate(test_df)
success_rate

11.678571428571429

In [11]:
avg_length, avg_return = utils.get_avg_length_reward(test_df)
avg_length, avg_return

(1.5399285714285715, -0.772)

In [12]:
acc, f1, roc_auc = utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc

(0.11678571428571428, 0.11539404043969742, 0.4976183828511908)

In [13]:
test_df.y_pred.unique()

array([1., 0., 4., 3., 2., 6., 7., 5.])